<a href="https://colab.research.google.com/github/KimGriezmann/Bigcontest_2021/blob/master/working_space_JaeHong/pre_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tqdm
import os
import matplotlib.pyplot as plt 
import seaborn as sns 

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
test_data = pd.read_csv('/content/drive/MyDrive/bigcontest/test_data.csv', encoding = 'euc-kr')

In [13]:
#누락된 출루율 계산
#타석(PA) 열 추가 / PA = 타수 + 볼넷 + 희비
#1루타(1_hit) 열 추가 / 1_hit = hit - 2_hit - 3_hit - homerun
#장타(XH) 열 추가 / XH = 2_hit + 3_hit + homerun

test_data['avg'] = round(test_data['hit'] / test_data['ab'], 3)
test_data['obp'] = round((test_data['hit'] + test_data['bb'] + test_data['hpp']) / (test_data['ab'] + test_data['bb'] + test_data['hpp'] + test_data['sf']), 3)
test_data['slg'] = round(test_data['tb'] / test_data['ab'], 3)
test_data['ops'] = round(test_data['obp'] + test_data['slg'], 3)

test_data.insert(7, 'pa', test_data['ab'] + test_data['bb'] + test_data['sf'])
test_data.insert(11, '1_hit', test_data['hit'] - test_data['2_hit'] - test_data['3_hit'] - test_data['homerun'])
test_data.insert(15, 'xh', test_data['2_hit'] + test_data['3_hit'] + test_data['homerun'])

In [14]:
test_data

,name,day,vs,result,bat_order,position,start_member,pa,ab,score,hit,1_hit,2_hit,3_hit,homerun,xh,tb,rbi,sb,cs,bb,hpp,ibb,so,gdp,sh,sf,avg,obp,slg,ops,pitch,avLI,RE24,WPA
0,강백호,04월 04일,한화,W 3:2,4,1B,1,4,3,0,2,2,0,0,0,0,2,0,2,0,1,0,0,0,0,0,0,0.667,0.750,0.667,1.417,16,1.14,0.89,0.122
1,강백호,04월 06일,LG,L 2:3,4,1B,1,8,7,0,2,2,0,0,0,0,2,0,2,0,1,0,0,2,0,0,0,0.286,0.375,0.286,0.661,30,2.07,-1.00,-0.183
2,강백호,04월 07일,LG,W 7:3,4,1B,1,13,11,0,5,5,0,0,0,0,5,1,2,0,2,0,0,2,0,0,0,0.455,0.538,0.455,0.993,47,1.43,1.75,0.330
3,강백호,04월 08일,LG,L 3:7,4,1B,1,17,15,0,7,7,0,0,0,0,7,1,2,0,2,0,0,3,1,0,0,0.467,0.529,0.467,0.996,64,0.89,-0.76,-0.102
4,강백호,04월 09일,@삼성,L 5:7,4,1B,1,22,20,0,10,9,0,1,0,1,12,4,2,0,2,0,0,4,2,0,0,0.500,0.545,0.600,1.145,85,0.94,1.88,-0.122
5,강백호,04월 10일,@삼성,L 6:7,4,RF,1,26,23,1,11,10,0,1,0,1,13,5,2,0,3,0,0,4,2,0,0,0.478,0.538,0.565,1.103,107,0.94,1.00,0.103
6,강백호,04월 11일,@삼성,L 2:4,4,RF,1,30,27,1,12,11,0,1,0,1,14,5,2,0,3,0,0,4,2,0,0,0.444,0.500,0.519,1.019,123,1.57,-0.20,-0.024
7,강백호,04월 13일,@두산,W 8:7,3,1B,1,35,32,1,13,12,0,1,0,1,15,5,2,0,3,0,0,5,3,0,0,0.406,0.457,0.469,0.926,141,1.31,-2.23,-0.100
8,강백호,04월 14일,@두산,L 1:3,3,1B,1,39,35,1,13,12,0,1,0,1,15,5,2,0,4,0,0,5,3,0,0,0.371,0.436,0.429,0.865,164,1.19,-0.30,-0.042


In [15]:
def cal_feature(data):
  babip = round((data['hit'] - data['homerun']) / (data['ab'] - data['so'] - data['homerun'] + data['sf']), 3) #babip
  pitch_pa = round(data['pitch'] / data['pa'], 2) #타석당 투구수
  cq = round((0.5 * data['hit'] + 0.3 * data['tb']) / (data['ab'] - data['so']), 3) #컨택 퀄리티
  b_k = round(data['bb'] / data['so'], 3) #볼삼비
  IsoP = data['slg'] - data['avg'] #순수장타율(장타율 - 타율) 
  IsoD = data['obp'] - data['avg'] #순수출루율(출루율 - 타율)
  psn = round((2 * data['homerun'] * data['sb']) / (data['homerun'] + data['sb']), 3) #호타준족수치
  hr_p = round(data['homerun'] / data['pa'], 3) #타석당 홈런비율
  kk_p = round(data['so'] / data['pa'], 3) #타석당 삼진비율
  bb_p = round(data['bb'] / data['pa'], 3) #타석당 볼넷비율
  xh_ab = round(data['xh'] / data['pa'], 3) #타석당 장타비율
  xh_h = round(data['xh'] / data['hit'], 3) #장타비율
  hr_xh = round(data['homerun'] / data['xh'], 3) #장타중 홈런비율
  pa_hr = round(data['pa'] / data['homerun'], 3) #홈런당 타석
  ab_hr = round(data['ab'] / data['homerun'], 3) #홈런당 타수
  ueqr = round((data['hit'] + data['tb'] + 1.5*(data['bb'] + data['hpp'] + data['sb']) + data['sh'] + data['sf'] - data['ibb']/2) / (data['ab'] + data['bb'] + data['hpp'] + data['sh'] + data['sf'] + data['cs'] + data['sb']), 3) #득점창출력
  woba = round((0.72 * data['bb'] + 0.75 * data['hpp'] + 0.9 * data['1_hit'] + 1.24 * data['2_hit'] + 1.56 * data['3_hit'] + 1.95 * data['homerun']) / (data['pa'] - data['ibb']), 3) #조정된 가중출루율

  data['babip'] = babip
  data['pitch/pa'] = pitch_pa
  data['b/k'] = b_k
  data['IsoP'] = IsoP
  data['IsoD'] = IsoD
  data['psn'] = psn
  data['hr%'] = hr_p
  data['kk%'] = kk_p
  data['bb%'] = bb_p
  data['xh/ab'] = xh_ab
  data['xh/h'] = xh_h
  data['hr/xh'] = hr_xh
  data['pa/hr'] = pa_hr
  data['ab/hr'] = ab_hr
  data['ueqr'] = ueqr
  data['wOBA'] = woba

  return data

In [16]:
test = cal_feature(test_data)

In [17]:
test.iloc[:,7:]

,pa,ab,score,hit,1_hit,2_hit,3_hit,homerun,xh,tb,rbi,sb,cs,bb,hpp,ibb,so,gdp,sh,sf,avg,obp,slg,ops,pitch,avLI,RE24,WPA,babip,pitch/pa,b/k,IsoP,IsoD,psn,hr%,kk%,bb%,xh/ab,xh/h,hr/xh,pa/hr,ab/hr,ueqr,wOBA
0,4,3,0,2,2,0,0,0,0,2,0,2,0,1,0,0,0,0,0,0,0.667,0.750,0.667,1.417,16,1.14,0.89,0.122,0.667,4.00,inf,0.000,0.083,0.0,0.0,0.000,0.250,0.000,0.000,NaN,inf,inf,1.417,0.630
1,8,7,0,2,2,0,0,0,0,2,0,2,0,1,0,0,2,0,0,0,0.286,0.375,0.286,0.661,30,2.07,-1.00,-0.183,0.400,3.75,0.500,0.000,0.089,0.0,0.0,0.250,0.125,0.000,0.000,NaN,inf,inf,0.850,0.315
2,13,11,0,5,5,0,0,0,0,5,1,2,0,2,0,0,2,0,0,0,0.455,0.538,0.455,0.993,47,1.43,1.75,0.330,0.556,3.62,1.000,0.000,0.083,0.0,0.0,0.154,0.154,0.000,0.000,NaN,inf,inf,1.067,0.457
3,17,15,0,7,7,0,0,0,0,7,1,2,0,2,0,0,3,1,0,0,0.467,0.529,0.467,0.996,64,0.89,-0.76,-0.102,0.583,3.76,0.667,0.000,0.062,0.0,0.0,0.176,0.118,0.000,0.000,NaN,inf,inf,1.053,0.455
4,22,20,0,10,9,0,1,0,1,12,4,2,0,2,0,0,4,2,0,0,0.500,0.545,0.600,1.145,85,0.94,1.88,-0.122,0.625,3.86,0.500,0.100,0.045,0.0,0.0,0.182,0.091,0.045,0.100,0.0,inf,inf,1.167,0.505
5,26,23,1,11,10,0,1,0,1,13,5,2,0,3,0,0,4,2,0,0,0.478,0.538,0.565,1.103,107,0.94,1.00,0.103,0.579,4.12,0.750,0.087,0.060,0.0,0.0,0.154,0.115,0.038,0.091,0.0,inf,inf,1.125,0.489
6,30,27,1,12,11,0,1,0,1,14,5,2,0,3,0,0,4,2,0,0,0.444,0.500,0.519,1.019,123,1.57,-0.20,-0.024,0.522,4.10,0.750,0.075,0.056,0.0,0.0,0.133,0.100,0.033,0.083,0.0,inf,inf,1.047,0.454
7,35,32,1,13,12,0,1,0,1,15,5,2,0,3,0,0,5,3,0,0,0.406,0.457,0.469,0.926,141,1.31,-2.23,-0.100,0.481,4.03,0.600,0.063,0.051,0.0,0.0,0.143,0.086,0.029,0.077,0.0,inf,inf,0.959,0.415
8,39,35,1,13,12,0,1,0,1,15,5,2,0,4,0,0,5,3,0,0,0.371,0.436,0.429,0.865,164,1.19,-0.30,-0.042,0.433,4.21,0.800,0.058,0.065,0.0,0.0,0.128,0.103,0.026,0.077,0.0,inf,inf,0.902,0.391


In [19]:
def make_percentage(data):
  data['hit'] = round(data['hit'] / data['pa'], 3)
  data['1_hit'] = round(data['1_hit'] / data['pa'], 3)
  data['xh'] = round(data['xh'] / data['pa'], 3)
  data['2_hit'] = round(data['2_hit'] / data['pa'], 3)
  data['3_hit'] = round(data['3_hit'] / data['pa'], 3)
  data['homerun'] = round(data['homerun'] / data['pa'], 3)
  data['tb'] = round(data['tb'] / data['pa'], 3)
  data['rbi'] = round(data['rbi'] / data['pa'], 3)
  data['sb'] = round(data['sb'] / data['pa'], 3)
  data['cs'] = round(data['cs'] / data['pa'], 3)
  data['bb'] = round(data['bb'] / data['pa'], 3)
  data['hpp'] = round(data['hpp'] / data['pa'], 3)
  data['ibb'] = round(data['ibb'] / data['pa'], 3)
  data['so'] = round(data['so'] / data['pa'], 3)
  data['gdp'] = round(data['gdp'] / data['pa'], 3)
  data['sh'] = round(data['sh'] / data['pa'], 3)
  data['sf'] = round(data['sf'] / data['pa'], 3)
  data['avLI'] = round(data['avLI'] / data['pa'], 3)
  data['RE24'] = round(data['RE24'] / data['pa'], 3)
  data['WPA'] = round(data['WPA'] / data['pa'], 3)

  return data


In [20]:
test2 = make_percentage(test)

In [21]:
test2.iloc[:,7:]

,pa,ab,score,hit,1_hit,2_hit,3_hit,homerun,xh,tb,rbi,sb,cs,bb,hpp,ibb,so,gdp,sh,sf,avg,obp,slg,ops,pitch,avLI,RE24,WPA,babip,pitch/pa,b/k,IsoP,IsoD,psn,hr%,kk%,bb%,xh/ab,xh/h,hr/xh,pa/hr,ab/hr,ueqr,wOBA
0,4,3,0,0.500,0.500,0.0,0.000,0.0,0.000,0.500,0.000,0.500,0.0,0.250,0.0,0.0,0.000,0.000,0.0,0.0,0.667,0.750,0.667,1.417,16,0.285,0.222,0.030,0.667,4.00,inf,0.000,0.083,0.0,0.0,0.000,0.250,0.000,0.000,NaN,inf,inf,1.417,0.630
1,8,7,0,0.250,0.250,0.0,0.000,0.0,0.000,0.250,0.000,0.250,0.0,0.125,0.0,0.0,0.250,0.000,0.0,0.0,0.286,0.375,0.286,0.661,30,0.259,-0.125,-0.023,0.400,3.75,0.500,0.000,0.089,0.0,0.0,0.250,0.125,0.000,0.000,NaN,inf,inf,0.850,0.315
2,13,11,0,0.385,0.385,0.0,0.000,0.0,0.000,0.385,0.077,0.154,0.0,0.154,0.0,0.0,0.154,0.000,0.0,0.0,0.455,0.538,0.455,0.993,47,0.110,0.135,0.025,0.556,3.62,1.000,0.000,0.083,0.0,0.0,0.154,0.154,0.000,0.000,NaN,inf,inf,1.067,0.457
3,17,15,0,0.412,0.412,0.0,0.000,0.0,0.000,0.412,0.059,0.118,0.0,0.118,0.0,0.0,0.176,0.059,0.0,0.0,0.467,0.529,0.467,0.996,64,0.052,-0.045,-0.006,0.583,3.76,0.667,0.000,0.062,0.0,0.0,0.176,0.118,0.000,0.000,NaN,inf,inf,1.053,0.455
4,22,20,0,0.455,0.409,0.0,0.045,0.0,0.045,0.545,0.182,0.091,0.0,0.091,0.0,0.0,0.182,0.091,0.0,0.0,0.500,0.545,0.600,1.145,85,0.043,0.085,-0.006,0.625,3.86,0.500,0.100,0.045,0.0,0.0,0.182,0.091,0.045,0.100,0.0,inf,inf,1.167,0.505
5,26,23,1,0.423,0.385,0.0,0.038,0.0,0.038,0.500,0.192,0.077,0.0,0.115,0.0,0.0,0.154,0.077,0.0,0.0,0.478,0.538,0.565,1.103,107,0.036,0.038,0.004,0.579,4.12,0.750,0.087,0.060,0.0,0.0,0.154,0.115,0.038,0.091,0.0,inf,inf,1.125,0.489
6,30,27,1,0.400,0.367,0.0,0.033,0.0,0.033,0.467,0.167,0.067,0.0,0.100,0.0,0.0,0.133,0.067,0.0,0.0,0.444,0.500,0.519,1.019,123,0.052,-0.007,-0.001,0.522,4.10,0.750,0.075,0.056,0.0,0.0,0.133,0.100,0.033,0.083,0.0,inf,inf,1.047,0.454
7,35,32,1,0.371,0.343,0.0,0.029,0.0,0.029,0.429,0.143,0.057,0.0,0.086,0.0,0.0,0.143,0.086,0.0,0.0,0.406,0.457,0.469,0.926,141,0.037,-0.064,-0.003,0.481,4.03,0.600,0.063,0.051,0.0,0.0,0.143,0.086,0.029,0.077,0.0,inf,inf,0.959,0.415
8,39,35,1,0.333,0.308,0.0,0.026,0.0,0.026,0.385,0.128,0.051,0.0,0.103,0.0,0.0,0.128,0.077,0.0,0.0,0.371,0.436,0.429,0.865,164,0.031,-0.008,-0.001,0.433,4.21,0.800,0.058,0.065,0.0,0.0,0.128,0.103,0.026,0.077,0.0,inf,inf,0.902,0.391
